**Task 1 Algorithmic Analysis K-Means Clustering with Real World Dataset**
First, download a simulated dataset: kmeans_data.zip from Modules->Datasets. Then,
implement the K-means algorithm from scratch. K-means algorithm computes the distance of a
given data point pair. Replace the distance computation function with Euclidean distance, 1-
Cosine similarity, and 1 – the Generalized Jaccard similarity (refer to:
https://www.itl.nist.gov/div898/software/dataplot/refman2/auxillar/jaccard.htm).


In [1]:
# import libraries

import numpy as np
from sklearn.metrics import pairwise_distances
import pandas as pd

In [2]:
data = pd.read_csv('./kmeans_data/data.csv')
labels_data = pd.read_csv('./kmeans_data/label.csv')

In [3]:
data.head

<bound method NDFrame.head of       0  0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  ...  0.658  0.659  \
0     0    0    0    0    0    0    0    0    0    0  ...      0      0   
1     0    0    0    0    0    0    0    0    0    0  ...      0      0   
2     0    0    0    0    0    0    0    0    0    0  ...      0      0   
3     0    0    0    0    0    0    0    0    0    0  ...      0      0   
4     0    0    0    0    0    0    0    0    0    0  ...      0      0   
...  ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...    ...    ...   
9994  0    0    0    0    0    0    0    0    0    0  ...      0      0   
9995  0    0    0    0    0    0    0    0    0    0  ...      0      0   
9996  0    0    0    0    0    0    0    0    0    0  ...      0      0   
9997  0    0    0    0    0    0    0    0    0    0  ...      0      0   
9998  0    0    0    0    0    0    0    0    0    0  ...      0      0   

      0.660  0.661  0.662  0.663  0.664  0.665  0.666  0.667  
0     

In [4]:
labels_data.head

<bound method NDFrame.head of       7
0     2
1     1
2     0
3     4
4     1
...  ..
9994  2
9995  3
9996  4
9997  5
9998  6

[9999 rows x 1 columns]>

In [5]:
class KMeans:
    def __init__(self, n_clusters, distance_metric='euclidean', max_iter=100):
        self.n_clusters = n_clusters
        self.distance_metric = distance_metric
        self.max_iter = max_iter
        self.centroids = None

    def initialize_centroids(self, X):
        np.random.seed(42)
        # Convert to numpy
        if isinstance(X, pd.DataFrame):
            X = X.to_numpy()
        random_indices = np.random.permutation(X.shape[0])[:self.n_clusters]
        self.centroids = X[random_indices]

    def compute_distance(self, X):
        if self.distance_metric == 'euclidean':
            return pairwise_distances(X, self.centroids, metric='euclidean')
        elif self.distance_metric == 'cosine':
            return 1 - pairwise_distances(X, self.centroids, metric='cosine')
        elif self.distance_metric == 'jaccard':
            # For Jaccard, handling the case where all features are zeros which can cause NaN distances
            return 1 - pairwise_distances(np.maximum(X, 0), np.maximum(self.centroids, 0), metric='jaccard')

    def assign_clusters(self, distances):
        return np.argmin(distances, axis=1)

    def update_centroids(self, X, labels):
        new_centroids = np.zeros(self.centroids.shape)
        for i in range(self.n_clusters):
            if np.any(labels == i):
                new_centroids[i] = X[labels == i].mean(axis=0)
            else:
                new_centroids[i] = self.centroids[i]  # Reuse the old centroid if no points are assigned
        return new_centroids

    def compute_sse(self, X, labels):
        distances = np.min(self.compute_distance(X), axis=1)
        return np.sum(distances**2)

    def fit(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.to_numpy()
        self.initialize_centroids(X)
        for _ in range(self.max_iter):
            distances = self.compute_distance(X)
            labels = self.assign_clusters(distances)
            new_centroids = self.update_centroids(X, labels)

            if np.allclose(self.centroids, new_centroids, atol=1e-4):
                break
            self.centroids = new_centroids

        return labels, self.compute_sse(X, labels)

In [6]:
n_clusters = len(np.unique(labels_data))
print(n_clusters)

10


**Q1. Run K-means clustering with Euclidean, Cosine and Jaccard similarity. Specify K= the
number of categorical values of y (the number of classifications). Compare the SSEs of
Euclidean-K-means, Cosine-K-means, Jaccard-K-means. Which method is better?**

In [7]:
# Run K-means with different distance metrics
results_final = {}
for metric in ['euclidean', 'cosine', 'jaccard']:
    kmeans_final = KMeans(n_clusters=n_clusters, distance_metric=metric)
    labels, sse = kmeans_final.fit(data)
    results_final[metric] = {"labels": labels, "sse": sse}
    print(labels)
results_final

[2 3 9 ... 4 8 7]
[2 2 3 ... 7 2 2]


/Users/gaurav/projects/DM/DM_CSE572/hw3/hw3/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/Users/gaurav/projects/DM/DM_CSE572/hw3/hw3/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/Users/gaurav/projects/DM/DM_CSE572/hw3/hw3/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/Users/gaurav/projects/DM/DM_CSE572/hw3/hw3/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/Users/gaurav/projects/DM/DM_CSE572/hw3/hw3/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2317: D

[0 0 0 ... 0 0 0]


/Users/gaurav/projects/DM/DM_CSE572/hw3/hw3/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


{'euclidean': {'labels': array([2, 3, 9, ..., 4, 8, 7]),
  'sse': 25321981136.02128},
 'cosine': {'labels': array([2, 2, 3, ..., 7, 2, 2]),
  'sse': 546.2471268877401},
 'jaccard': {'labels': array([0, 0, 0, ..., 0, 0, 0]),
  'sse': 550.3072335867188}}

Based on the SSE values derived:

Euclidean-K-means: SSE = 25321981136.02128
Cosine-K-means: SSE = 546.25
Jaccard-K-means: SSE = 550.31

The **Cosine-K-means** method shows the lowest SSE, indicating it performs the best in terms of cluster compactness for dataset.

Therefore, based on observations **Cosine-K-means** is better method.

**Q2**: Compare the accuracies of Euclidean-K-means Cosine-K-means, Jaccard-K-means. First,
label each cluster using the majority vote label of the data points in that cluster. Later, compute
the predictive accuracy of Euclidean-K-means, Cosine-K-means, Jaccard-K-means. Which metric is better? (10 points)


In [8]:
def label_clusters(labels, true_labels, k):
    cluster_labels = {}
    for i in range(k):
        # Find the most common true label in each cluster
        labels_in_cluster = true_labels[labels == i]
        if len(labels_in_cluster) == 0:
            continue
        most_common = np.bincount(labels_in_cluster).argmax()
        cluster_labels[i] = most_common
    return cluster_labels

In [9]:
def compute_accuracy(labels, true_labels, cluster_labels):
    correct_predictions = 0
    for label, true_label in zip(labels, true_labels):
        if cluster_labels[label] == true_label:
            correct_predictions += 1
    return correct_predictions / len(true_labels)

In [10]:
# If labels_data is a pandas DataFrame column, ensure it's one-dimensional
if labels_data.ndim > 1:
    labels_data = labels_data.squeeze()
# Convert true labels to numeric form if they are not already
true_labels_numeric = pd.factorize(labels_data)[0]

# Label each cluster using majority vote for each distance measure
cluster_labels_euclidean = label_clusters(results_final['euclidean']['labels'], true_labels_numeric, n_clusters)
cluster_labels_cosine = label_clusters(results_final['cosine']['labels'], true_labels_numeric, n_clusters)
cluster_labels_jaccard = label_clusters(results_final['jaccard']['labels'], true_labels_numeric, n_clusters)

# Compute the predictive accuracy for each distance measure
accuracy_euclidean = compute_accuracy(results_final['euclidean']['labels'], true_labels_numeric, cluster_labels_euclidean)
accuracy_cosine = compute_accuracy(results_final['cosine']['labels'], true_labels_numeric, cluster_labels_cosine)
accuracy_jaccard = compute_accuracy(results_final['jaccard']['labels'], true_labels_numeric, cluster_labels_jaccard)

print("Accuracy Euclidean:", accuracy_euclidean)
print("Accuracy Cosine:", accuracy_cosine)
print("Accuracy Jaccard:", accuracy_jaccard)

Accuracy Euclidean: 0.6004600460046005
Accuracy Cosine: 0.23932393239323932
Accuracy Jaccard: 0.11351135113511351


Based on the results derived, 

Accuracy Euclidean: 0.6004600460046005
Accuracy Cosine: 0.23932393239323932
Accuracy Jaccard: 0.11351135113511351

Euclidean is better.

**Q3:** Set up the same stop criteria: “when there is no change in centroid position OR when the
SSE value increases in the next iteration OR when the maximum preset value (e.g., 500, you
can set the preset value by yourself) of iteration is complete”, for Euclidean-K-means, Cosine-K-
means, Jaccard-K-means. Which method requires more iterations and times to converge? (10
points)


In [11]:
class KMeans_Convergence:
    def __init__(self, n_clusters, distance_metric='euclidean', max_iter=500):
        self.n_clusters = n_clusters
        self.distance_metric = distance_metric
        self.max_iter = max_iter
        self.centroids = None

    def initialize_centroids(self, X):
        np.random.seed(42)
        if isinstance(X, pd.DataFrame):
            X = X.to_numpy()
        random_indices = np.random.permutation(X.shape[0])[:self.n_clusters]
        self.centroids = X[random_indices]

    def compute_distance(self, X):
        if self.distance_metric == 'euclidean':
            return pairwise_distances(X, self.centroids, metric='euclidean')
        elif self.distance_metric == 'cosine':
            return 1 - pairwise_distances(X, self.centroids, metric='cosine')
        elif self.distance_metric == 'jaccard':
            return 1 - pairwise_distances(np.maximum(X, 0), np.maximum(self.centroids, 0), metric='jaccard')

    def assign_clusters(self, distances):
        return np.argmin(distances, axis=1)

    def update_centroids(self, X, labels):
        new_centroids = np.zeros(self.centroids.shape)
        for i in range(self.n_clusters):
            if np.any(labels == i):
                new_centroids[i] = X[labels == i].mean(axis=0)
        return new_centroids

    def compute_sse(self, X, labels):
        distances = np.min(self.compute_distance(X), axis=1)
        return np.sum(distances**2)

    def fit(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.to_numpy()
        self.initialize_centroids(X)
        previous_sse = None
        for iteration in range(self.max_iter):
            distances = self.compute_distance(X)
            labels = self.assign_clusters(distances)
            new_centroids = self.update_centroids(X, labels)
            sse = self.compute_sse(X, labels)

            if np.allclose(self.centroids, new_centroids, atol=1e-4) or (previous_sse is not None and sse > previous_sse):
                break
            self.centroids = new_centroids
            previous_sse = sse

        return labels, sse, iteration + 1


In [12]:
import time
# Run KMeans_Convergence with different distance metrics
results_convergence = {}
for metric in ['euclidean', 'cosine', 'jaccard']:
    kmeans_final = KMeans_Convergence(n_clusters=n_clusters, distance_metric=metric)
    start_time = time.time()  # Start time
    labels, sse, iterations = kmeans_final.fit(data)
    end_time = time.time()  # End time
    time_taken = end_time - start_time  # Calculate time taken
    results_convergence[metric] = {"labels": labels, "sse": sse, "iterations": iterations, "time_taken": time_taken}
results_convergence

/Users/gaurav/projects/DM/DM_CSE572/hw3/hw3/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/Users/gaurav/projects/DM/DM_CSE572/hw3/hw3/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/Users/gaurav/projects/DM/DM_CSE572/hw3/hw3/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/Users/gaurav/projects/DM/DM_CSE572/hw3/hw3/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


{'euclidean': {'labels': array([2, 3, 9, ..., 4, 8, 7]),
  'sse': 25321981136.02128,
  'iterations': 66,
  'time_taken': 6.093734979629517},
 'cosine': {'labels': array([1, 1, 1, ..., 1, 1, 1]),
  'sse': 0.0,
  'iterations': 500,
  'time_taken': 46.23327112197876},
 'jaccard': {'labels': array([8, 5, 8, ..., 0, 5, 8]),
  'sse': 462.04767139500063,
  'iterations': 2,
  'time_taken': 0.6264011859893799}}

Based on the results, 

**cosine takes more time and iterations to converge**

**Q4:** Compare the SSEs of Euclidean-K-means Cosine-K-means, Jaccard-K-means with respect to
the following three terminating conditions: (10 points)
• when there is no change in centroid position
• when the SSE value increases in the next iteration
• when the maximum preset value (e.g., 100) of iteration is complete


In [13]:
import numpy as np
from sklearn.metrics import pairwise_distances
import time

class KMeansConditional:
    def __init__(self, n_clusters, distance_metric='euclidean', max_iter=100):
        self.n_clusters = n_clusters
        self.distance_metric = distance_metric
        self.max_iter = max_iter
        self.centroids = None

    def initialize_centroids(self, X):
        np.random.seed(42)
        if isinstance(X, pd.DataFrame):
            X = X.to_numpy()
        random_indices = np.random.permutation(X.shape[0])[:self.n_clusters]
        self.centroids = X[random_indices]

    def compute_distance(self, X):
        if self.distance_metric == 'euclidean':
                return pairwise_distances(X, self.centroids, metric='euclidean')
        elif self.distance_metric == 'cosine':
            return 1 - pairwise_distances(X, self.centroids, metric='cosine')
        elif self.distance_metric == 'jaccard':
            return 1 - pairwise_distances(np.maximum(X, 0), np.maximum(self.centroids, 0), metric='jaccard')

    def assign_clusters(self, distances):
        return np.argmin(distances, axis=1)

    def update_centroids(self, X, labels):
        new_centroids = np.zeros(self.centroids.shape)
        for i in range(self.n_clusters):
            cluster_points = X[labels == i]
            if len(cluster_points) > 0:
                new_centroids[i] = cluster_points.mean(axis=0)
        return new_centroids

    def compute_sse(self, X, labels):
        distances = np.min(self.compute_distance(X), axis=1)
        return np.sum(distances ** 2)

    def fit(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.to_numpy()
        self.initialize_centroids(X)
        previous_sse = None
        no_change_count = 0
        for iteration in range(self.max_iter):
            distances = self.compute_distance(X)
            labels = self.assign_clusters(distances)
            new_centroids = self.update_centroids(X, labels)
            sse = self.compute_sse(X, labels)

            if np.allclose(self.centroids, new_centroids, atol=1e-4):
                no_change_count += 1
            else:
                no_change_count = 0  # Reset count if there's a change

            if no_change_count > 1 or (previous_sse is not None and sse > previous_sse):
                break

            self.centroids = new_centroids
            previous_sse = sse

        return labels, sse, iteration + 1

In [14]:
results_conditional = {}
for metric in ['euclidean', 'cosine', 'jaccard']:
    kmeans = KMeansConditional(n_clusters=n_clusters, distance_metric=metric, max_iter=100)
    labels, sse, iterations = kmeans.fit(data)
    results_conditional[metric] = {"SSE": sse, "iterations": iterations}

results_conditional

/Users/gaurav/projects/DM/DM_CSE572/hw3/hw3/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/Users/gaurav/projects/DM/DM_CSE572/hw3/hw3/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/Users/gaurav/projects/DM/DM_CSE572/hw3/hw3/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/Users/gaurav/projects/DM/DM_CSE572/hw3/hw3/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


{'euclidean': {'SSE': 25321981136.02128, 'iterations': 67},
 'cosine': {'SSE': 0.0, 'iterations': 100},
 'jaccard': {'SSE': 462.04767139500063, 'iterations': 2}}

**Q5:** What are your summary observations or takeaways based on your algorithmic analysis? (5
points)

**Answer-:**
Euclidean has very high SSE, cosine is nearly 0 and jaccard is moderate. This shows that there is significant difference in compactness of clusters using different methods.
Euclidean has moderate iterations, cosine takes a long time while jaccard converges very quickly. 
Euclidean has better accuracy than Cosine and Jaccard.
Cosine is time consuming and Jaccard is the fastest. But Euclidean yields better results.
Euclidean appears to be more suitable for the dataset than any other methods.

**In Conclusion, despite higher SSE, Euclidean offers better results, moderate convergence and iterations, making it the best choice for this dataset.**
